In [5]:
import numpy as np 
import pandas as pd 
import mlflow
import mlflow.lightgbm
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from sklearn.model_selection import TimeSeriesSplit
from sktime.performance_metrics.forecasting import (MeanAbsoluteScaledError, 
    MeanAbsolutePercentageError, MeanAbsoluteError, MeanSquaredError)
from lightgbm import LGBMRegressor

import optuna
from optuna.integration.mlflow import MLflowCallback
import joblib

from typing import Optional, Dict, Tuple, Literal
from enefit_challenge.models.forecaster import Forecaster

import warnings
warnings.filterwarnings('ignore')

TRACKING_URI = "http://127.0.0.1:5000/" # local tracking URI -> launch mlflow before training 


In this notebook we're going to do two things:
1. inform the LightGBMForecaster of the uselessness of some features, according to previous model's iteration feature importance 
2. define a new subset of parameters to run experiments on, according to mlflow's [parallel coordinates chart](../../assets/lgbm_par_coordinates.png)


In [6]:
from enefit_challenge.utils.dataset import load_enefit_training_data
not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id']
cat_columns = ['county', 'product_type']
to_drop_cols = [
    '10_metre_u_wind_component_mean_f',
    '10_metre_v_wind_component_min_f',
    'cloudcover_low_mean_f',
    'dayofweek_sine',
    'direct_solar_radiation_max_f',
    'eic_count',
    'euros_per_mwh', # not sure about this one
    'hour_sine',
    'temperature_min_f',
    'total_precipitation_max_f',
    'week_sine',
    '10_metre_u_wind_component_min_f',
    '10_metre_v_wind_component_max_f',
    '10_metre_v_wind_component_std_f',
    'cloudcover_high_mean_f',
    'cloudcover_high_std_f',
    'cloudcover_low_min_f',
    'cloudcover_low_std_f',
    'cloudcover_mid_std_f',
    'cloudcover_total_std_f',
    'county_12', # find a way to drop
    'county_3', # find a way to drop
    'county_9', # find a way to drop
    'direct_solar_radiation_min_f',
    'direct_solar_radiation_std_f',
    'highest_price_per_mwh', # not sure about this one
    # 'installed_capacity', #this one should alspo be dropped?
    'month_cosine',
    'product_type_3', # find a way to drop
    'snowfall_max_f',
    'temperature_std_f',
    'total_precipitation_min_f',
    'total_precipitation_std_f',
    # 'year'
]
df_train = load_enefit_training_data()

In [9]:
class LightGBMForecaster(Forecaster):
    """
        Implementation of a Forecaster using `LGBMRegressor` as base model, 
        `optuna` for hyperparameters optimization and `mlflow` as backend to track experiments
        and register best-in-class model for time series prediction.
    """
    def __init__(self)-> None:
        self.tracking_uri = mlflow.set_tracking_uri(TRACKING_URI)
        pass

    def fit_model(
        self,  
        X:pd.DataFrame,
        y:pd.Series,
        params:Optional[Dict]=None,
    ) -> LGBMRegressor:
        """
        Trains a `LGBMRegressor`

        -------     
        params:
        -------
        `X`:`pd.DataFrame`
            Features to use for fitting
        `y`:`pd.Series`
            Target variable
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        -------     
        returns:
        -------
        fitted `LGBMRegressor`
        """
        model = LGBMRegressor(
            n_estimators=100, 
            objective='regression',
        )
        if params:
            model.set_params(**params)

        model.fit(X, y)
    
        return model
    
    def fit_and_test_fold(
        self, 
        params:Dict,
        X: pd.DataFrame, 
        y: pd.Series, 
        year_month_train, 
        year_month_test,
        experiment_name: str="lightgbm",
        artifact_path: str="lightgbm_model",
        metrics: list=["mae"]
    ) -> float:
        """
        Used for cross validation on different time splits; 
        also in charge of logging every experiment run / study trial into the backend.
        """
        
        first_dates_month = pd.to_datetime(X[['year', 'month']].assign(day=1))
        train_index = first_dates_month.isin(year_month_train)
        test_index = first_dates_month.isin(year_month_test)

        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]

        # fit model on training data
        mlflow.lightgbm.autolog(log_models=False, log_datasets=False)
        model = self.fit_model(
            X_train, 
            y_train, 
            params
        )
        
        # generate predictions
        y_test_pred = model.predict(X_test)
        self.signature = infer_signature(X_train, y_test_pred)
        MAE = MeanAbsoluteError()
        mae = MAE(y_test, y_test_pred)
        MASE = MeanAbsoluteScaledError()
        mase = MASE(y_test, y_test_pred, y_train=y_train)
        MAPE = MeanAbsolutePercentageError()
        mape = MAPE(y_test, y_test_pred)
        MSE = MeanSquaredError()
        mse = MSE(y_test, y_test_pred)
        RMSE = MeanSquaredError(square_root=True)
        rmse = RMSE(y_test, y_test_pred)

        mlflow.lightgbm.log_model(
            model, 
            artifact_path=artifact_path,
            signature=self.signature
        )
        mlflow.log_params(params)

        return mae, mase, mse, rmse, mape

    def train_model(
        self, 
        train_df: pd.DataFrame, 
        target_col: str,
        model_name: str,
        exclude_cols: list=[],
        categorical_features: list=[],
        experiment_name: str="lightgbm",
        artifact_path: str="lightgbm_model",
        params: Optional[Dict]=None
    ) -> None:
        """ 
        Takes an instance of `LGBMRegressor` model and tracks the hyperparameter tuning
        experiment on training set using `mlflow` and `optuna`.  
        Registers the best version of the model according to a specified metric (to be implemented).
        
        -------     
        params:
        -------
        `experiment_name`: `str`
            the name of the experiment used to store runs in mlflow, 
            as well as the name of the optuna study
        `model_name`: `str`
            the name the final model will have in the registry
        `train_df`: `pd.DataFrame`
            the training data for the model.
        `target_col`: `str`
            the time-series target column
        `exclude_cols`: `list`  
            columns in dataset that should not be used
        `categorical_features`: `list`
            list of categorical features in the dataset
        `artifact_path`: `str`
            the path pointing to the mlflow artifact
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        """
        self.model_name = model_name

        if len(categorical_features) > 0: 
           train_df = pd.get_dummies(train_df, columns=categorical_features)

        X = train_df.drop([target_col] + exclude_cols, axis=1)
        y = train_df[target_col]
        # unique year-month combinations -> to be used in cross-validation
        timesteps = np.sort(np.array(
            pd.to_datetime(X[['year', 'month']].assign(day=1)).unique().tolist()
        ))
        
        # define mlflow callback Handler for optuna 
        mlflc = MLflowCallback(
            metric_name=["MAE"]
        )
    
        @mlflc.track_in_mlflow() # decorator to allow mlflow logging
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 200, log=True),
                'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt"]),
                'learning_rate': trial.suggest_float('eta', 0.01, 0.2,log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 50, log=True),
                'max_depth': trial.suggest_int('max_depth', 1, 25, log=True),
                'min_child_weight': trial.suggest_float('min_child_weight', 0.01, 0.1, log=True),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1, log=True),
                'subsample': trial.suggest_float("subsample", 0.5, 0.85, log=True),
                'reg_alpha': trial.suggest_float('alpha', 1e-3, 0.2, log=True), #L1
                'reg_lambda': trial.suggest_float('lambda', 2, 10.0, log=True) #L2
            }
            cv = TimeSeriesSplit(n_splits=3) # cross validation
            cv_mae = [None]*3
            cv_mase = [None]*3
            cv_mse = [None]*3
            cv_rmse = [None]*3
            cv_mape = [None]*3
            for i, (train_index, test_index) in enumerate(cv.split(timesteps)):
                cv_mae[i], cv_mase[i], cv_mse[i], cv_rmse[i], cv_mape[i] = self.fit_and_test_fold(
                    params,
                    X, 
                    y, 
                    timesteps[train_index], 
                    timesteps[test_index]
                )
            trial.set_user_attr('split_mae', cv_mae)
            trial.set_user_attr('split_mase', cv_mase)
            trial.set_user_attr('split_mse', cv_mse)
            trial.set_user_attr('split_rmse', cv_rmse)
            trial.set_user_attr('split_mape', cv_mape)

            mlflow.log_metrics(
                {
                    "MAE":np.mean(cv_mae),
                    "MASE": np.mean(cv_mase),
                    "MSE": np.mean(cv_mse),
                    "RMSE":np.mean(cv_rmse),
                    "MAPE":np.mean(cv_mape)
                }
            )
            return np.mean(cv_mae) 

        
        sampler = optuna.samplers.TPESampler(
            n_startup_trials=10, 
            seed=0
        )

        self.study = optuna.create_study(
            directions=['minimize'],
            sampler=sampler,
            study_name=experiment_name
        )

        self.study.optimize(objective, n_trials=100, timeout= 7200, callbacks=[mlflc]) 
        
        # # search for the best run at the end of the experiment # not implemented now bc of callback bug
        # best_run = mlflow.search_runs(max_results=1,order_by=["metrics.MAE"]).run_id
        # # register new model version in mlflow
        # self.result = mlflow.register_model(
        #     model_uri=f"runs:/{best_run}/{artifact_path}",
        #     name=self.model_name
        # )

    def forecast(
        self, 
        input_data: pd.DataFrame,
        use_best_from_run: bool=True,
        use_env_model: Literal["Staging", "Production", None]=None,
        use_version: int=None
        ) -> pd.DataFrame:
        """ 
        Fetches a version of the model from the mlflow backend and uses it
        to perform prediction on new input data.  
        What version is used depends on params settings, 
        defaults to using the best version from the last experiment run (currently not implemented). 
        -------     
        params:
        -------
        `input_data`: `pd.DataFrame`
            the input data for prediction,
              must have the same schema as what's in the model's signature.
        `use_best_from_run`: `bool=True`      
            use the best model from the current series of iterations, defaults to True
        `use_env_model`: `Literal["Staging", "Production", None]=None`
            use model from a given mlflow environment, defaults to None.  
            Said model might come from past iterations, depending on what you decide in the UI
        `use_version`: `int=None`
            use a previously trained version of the model. 
            Said version must have been registered from a previous iteration,  
            either by the UI or with mlflow's API
        """
        if use_best_from_run:
            # not implemented now bc of callback bug
            use_prod_model=None
            use_version=None
        
            # model = mlflow.pyfunc.load_model(
            #     model_uri=f"models:/{self.model_name}/{self.result.version}"
            # )
            # y_pred = model.predict(input_data)
            # return y_pred
        
        if use_env_model is not None:
            use_version = None

            model = mlflow.pyfunc.load_model(
                # get registered model in given environment
                model_uri=f"models:/{self.model_name}/{use_env_model}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        if use_version is not None:
            # get specific registered version of model
            model = mlflow.pyfunc.load_model(
                model_uri=f"models:/{self.model_name}/{use_version}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        
        if (not use_best_from_run) & (use_env_model is None) & (use_version is None):
            return ValueError(
                    "You must specify which kind of LightGBMForecaster you intend to use for prediction"
            )

In [10]:
lgbf = LightGBMForecaster()

lgbf.train_model(
    train_df=df_train,
    target_col="target",
    model_name="lightgbm_enefit",
    exclude_cols=not_feature_columns+to_drop_cols,
    categorical_features=cat_columns
)

[I 2023-12-09 17:13:20,855] A new study created in memory with name: lightgbm


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015442 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8120
[LightGBM] [Info] Number of data points in the train set: 551778, number of used features: 59
[LightGBM] [Info] Start training from score 242.074828
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[I 2023-12-09 17:13:37,059] Trial 0 finished with value: 61.177352286648954 and parameters: {'n_estimators': 107, 'boosting_type': 'gbdt', 'eta': 0.0852084800119005, 'num_leaves': 35, 'max_depth': 4, 'min_child_weight': 0.02652496376759476, 'colsample_bytree': 0.7823543585516992, 'subsample': 0.630683552411224, 'alpha': 0.11271881751387774, 'lambda': 9.431947170347655}. Best is trial 0 with value: 61.177352286648954.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014379 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8120
[LightGBM] [Info] Number of data points in the train set: 551778, number of used features: 59
[LightGBM] [Info] Start training from score 242.074828
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[I 2023-12-09 17:13:52,556] Trial 1 finished with value: 67.26341474725164 and parameters: {'n_estimators': 85, 'boosting_type': 'gbdt', 'eta': 0.10716624725460688, 'num_leaves': 32, 'max_depth': 5, 'min_child_weight': 0.08425518534432111, 'colsample_bytree': 0.5252353993848682, 'subsample': 0.523659344156774, 'alpha': 0.0011130716950781927, 'lambda': 7.638468512687414}. Best is trial 0 with value: 61.177352286648954.


...

[I 2023-12-09 17:42:31,750] Trial 97 finished with value: 54.683655777467926 and parameters: {'n_estimators': 116, 'boosting_type': 'gbdt', 'eta': 0.11965596343201605, 'num_leaves': 42, 'max_depth': 15, 'min_child_weight': 0.0846682919551434, 'colsample_bytree': 0.9505592944575407, 'subsample': 0.7201404806358864, 'alpha': 0.1178006052309124, 'lambda': 3.6171548476587025}. Best is trial 65 with value: 52.64828710800964.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017956 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8120
[LightGBM] [Info] Number of data points in the train set: 551778, number of used features: 59
[LightGBM] [Info] Start training from score 242.074828
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036042 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8332
[LightGBM] [Info] Number of data points in the train set: 1042444, number of used features: 59
[LightGBM] [Info] Start training from score 247.723599
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061184 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is no

[I 2023-12-09 17:42:48,074] Trial 98 finished with value: 54.897558551442934 and parameters: {'n_estimators': 107, 'boosting_type': 'gbdt', 'eta': 0.09271633527711659, 'num_leaves': 35, 'max_depth': 17, 'min_child_weight': 0.07518551671771148, 'colsample_bytree': 0.8500943412974793, 'subsample': 0.6817373547460969, 'alpha': 0.15761417092787844, 'lambda': 3.8835042628320258}. Best is trial 65 with value: 52.64828710800964.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018935 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8120
[LightGBM] [Info] Number of data points in the train set: 551778, number of used features: 59
[LightGBM] [Info] Start training from score 242.074828
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036768 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8332
[LightGBM] [Info] Number of data points in the train set: 1042444, number of used features: 59
[LightGBM] [Info] Start training from score 247.723599
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058669 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is no

[I 2023-12-09 17:43:04,008] Trial 99 finished with value: 55.361201007763164 and parameters: {'n_estimators': 101, 'boosting_type': 'gbdt', 'eta': 0.13143187502796172, 'num_leaves': 39, 'max_depth': 21, 'min_child_weight': 0.06694194952658253, 'colsample_bytree': 0.9096881361949644, 'subsample': 0.6689026778082572, 'alpha': 0.1402891506098696, 'lambda': 3.3222213511693703}. Best is trial 65 with value: 52.64828710800964.
